# 1) Ingesta
# Leer los archivos de datos que sean .sv, .ndjson o .jsonl
files = sorted(DATA.glob("*.csv")) + sorted(DATA.glob("*.ndjson")) + sorted(DATA.glob("*.jsonl"))
raw = [] # Raw es igual a:
for f in files:  #Por cada archivo en files.
    if f.suffix.lower() == ".csv": #si la extension es .csv
        df = pd.read_csv(f, dtype=str) #Lee el archivo y lo pasa texto palno (str)
    else:  # ndjson/jsonl (son lo mismo) tambien lo pasa a texto plano. En teoría esta parte no hace falta porque no hay archivos de ese tipo.
        df = pd.read_json(f, lines=True, dtype=str)
    df["_source_file"] = f.name #Agrega una columna con el nombre del archivo.
    df["_ingest_ts"] = datetime.now(timezone.utc).isoformat() #Agrega una columna con la fecha y hora de ingesta.
    raw.append(df) #Agrega el dataframe a la lista raw.

if raw: #Si raw tiene datos
    raw_df = pd.concat(raw, ignore_index=True) #Concatena (los pone seguidos) todos los dataframes en uno solo.
else: #Si raw esta vacio
    cols = ["fecha","id_cliente","id_producto","unidades","precio_unitario","_source_file","_ingest_ts"] #Define las columnas que debe tener el dataframe.
    raw_df = pd.DataFrame(columns=cols) #Crea un dataframe vacio con esas columnas.

print(f"Ingested {len(raw_df)} rows from {len(files)} files.")
